In [15]:
#!/usr/bin/env python

This examples shows how we can approximate the eigenvectors of a kernel matrix by combining RFF and nystrom

By using RFF, we can get the feature map as computed by RFF directly. This allows us to obtain the eigenfunction in RKHS via<br>
1. Given $\psi$ as the feature map, compute the integral operator<br>
$$ T_n = \frac{1}{n} \sum_{i=1}^n \psi(x) \psi(x)^T $$<br>
2. Find the eigenfunctions<br>
$$ [\Phi, \Sigma] = eig(T_n) $$<br>
$$ \Sigma = diag(\sigma_1, \sigma_2, ..., ) $$<br>
$$ \Phi = [\phi_1, \phi_2, ...]$$<br>
3. We next use the eigenfunctions $\Phi$ to approximate the eigenvector of the kernel matrix<br>
The key insight is that given an approximation of the eigenfunction $\phi_i$, the corresponding eigenvector $u_i$ of the kernel matrix K is <br>
$$u_i = \frac{1}{\sqrt{\sigma_i}} \Psi_n \phi_i$$<br>
$$\Psi_n = \begin{bmatrix} \psi(x_1)^T \\ \psi(x_2)^T \\ ... \\ \psi(x_n) \end{bmatrix} $$<br>
$$U = \Psi_n \Phi^T \Sigma$$

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from sklearn.utils import shuffle
from sklearn.kernel_approximation import Nystroem
from sklearn.kernel_approximation import RBFSampler
from tools import *

Initialize all the setting

In [17]:
X = csv_load('../dataset/wine.csv', shuffle_samples=True)
q = 30				# size of submatrix A
n = X.shape[0]		# number of total samples
γ = get_rbf_γ(X)	# γ used for the gaussian kerenl

Use rff nystrom to approximate K eigenvectors

In [18]:
rbf_feature = RBFSampler(gamma=γ, random_state=1, n_components=100)
Ψ = rbf_feature.fit_transform(X)
Tn = Ψ.T.dot(Ψ)								# unscaled integral operator

Note that the approximation is running an eigendecomposition on 100x100 matrix instead of 178 and 178<br>
Obviously, the more samples we use the more accurate it would be.

In [19]:
[σs,V] = np.linalg.eig(Tn)					
Σ = np.diag(1/np.sqrt(σs[0:10]))
Φ = V[:, 0:10]
Ū = Ψ.dot(Φ).dot(Σ)		# Note: U bar above implies approximating U actual

Use the kernel itself to get the actual eigenvectors

In [20]:
K = sklearn.metrics.pairwise.rbf_kernel(X, gamma=γ)
[Λ, U] = np.linalg.eig(K)	# compute the "actual" eigenvectors on a 178 x 178 matrix

Print a portion the two eigenvectors at two locations

In [21]:
print_two_matrices_side_by_side(U[0:20, 0:3], Ū[0:20, 0:3], title1='Actual eigenvectors', title2='Approximated Eigenvectors')
print_two_matrices_side_by_side(U[80:100, 4:8], Ū[80:100, 4:8], title1='Actual eigenvectors', title2='Approximated Eigenvectors')

   Actual eigenvectors   	Approximated Eigenvectors
[-0.0424 -0.1298 -0.026 ]	[-0.0557  0.122  -0.0127]
[-0.0879  0.0552 -0.0449]	[-0.0823 -0.063  -0.0481]
[-0.081   0.0644 -0.0766]	[-0.0756 -0.0713 -0.0742]
[-0.0348 -0.1319 -0.064 ]	[-0.0491  0.1262 -0.0544]
[-0.0752  0.0672 -0.0929]	[-0.0696 -0.0734 -0.0867]
[-0.0862  0.0578 -0.0531]	[-0.0799 -0.0655 -0.0543]
[-0.0178 -0.1113 -0.139 ]	[-0.0315  0.1168 -0.1432]
[-0.0912  0.0465 -0.0219]	[-0.0869 -0.055  -0.0257]
[-0.0904  0.049  -0.0281]	[-0.086  -0.0571 -0.0313]
[-0.0879  0.0555 -0.0457]	[-0.0831 -0.0632 -0.0476]
[-0.0823 -0.0522  0.0888]	[-0.0861  0.0403  0.0943]
[-0.0802  0.0648 -0.0789]	[-0.0752 -0.0716 -0.0764]
[-0.0088 -0.0759 -0.1412]	[-0.0204  0.089  -0.1486]
[-0.0926 -0.0013  0.0577]	[-0.0914 -0.0088  0.0555]
[-0.0857 -0.0397  0.0864]	[-0.0876  0.0289  0.0893]
[-0.0581  0.0647 -0.1141]	[-0.0514 -0.0688 -0.0994]
[-0.0227 -0.1216 -0.1212]	[-0.0367  0.1231 -0.1199]
[-0.0024 -0.0294 -0.0795]	[-0.0115  0.0378 -0.0608]
[-0.0848  0.

In [22]:
avg_error = np.sum(np.absolute(U[:,0:10] - Ū))/(n*10)
jupyter_print('The average absolute error with RFF Nystrom of each element is %f'% avg_error)